# CNN implementation

In [3]:
import os

#os.chdir("/drive/MyDrive/data")

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

from torchvision.transforms import Compose, ToTensor, RandomAffine, RandomHorizontalFlip, RandomVerticalFlip, ColorJitter
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

import random
import utils

plt.rcParams['figure.figsize'] = (17, 5)

sr = 22050

In [4]:
def saveheavy(a, name, n):  #functions to save havy dataframes in multiple files
    l= len(a)
    if(n>1):
        for i in range(n-1):
            a_i = a[int((l/n)*i):int((l/n)*(i+1))]
            np.save(f'{name}_{i+1}.npy', a_i)
    a_i = a[int((l/n)*(n-1)):l]
    np.save(f'{name}_{n}.npy', a_i)
    
def readheavy(name, n, Dir):
    a = np.array([0,0])
    for i in range(n):
        new_a = np.load(f'Data/{Dir}/{name}_{i+1}.npy', allow_pickle = True)
        a = np.vstack([a, new_a])
    return a[1:]

In [5]:
#Restore Datasets

test = readheavy('test', 2, 'Audio')
training = readheavy('training', 16, 'Audio')
validation = readheavy('validation', 2, 'Audio')

MemoryError: 

In [4]:
training.shape

(6394, 2)

In [5]:
def get_stft(a):
    for j in range(len(a)):
        audio = a[j, 0]
        stft = np.abs(librosa.stft(audio, n_fft=1024, hop_length=512))
        a[j ,0] = stft
    return a

def get_mel(a):
    for j in range(len(a)):
        audio = a[j,0]
        stft = np.abs(librosa.stft(audio, n_fft=1024, hop_length=512))
        mel = mel = librosa.feature.melspectrogram(sr=sr, S=stft**2)
        a[j,0] = mel
    return a

In [6]:
#Now I convert Audio into stft data

test = get_stft(test)

In [7]:
validation = get_stft(validation)
training = get_stft(training)

In [8]:
def clip_stft(a, n_samples):
    a_clip = np.array([0,0])
    for j in range(len(a)):
        full = a[j, 0].T
        n=0
        while (n<(len(full)-n_samples)):
            clip = full[n: (n+n_samples)]
            y = a[j, 1]
            new_row = np.array([clip, y], dtype=object)
            a_clip = np.vstack([a_clip, new_row])
            n+=int(n_samples/2)
    return a_clip[1:]


def clip_audio(df, n_samples):
    a_clip = np.array([0,0])
    for j in range(len(a)):
        full = df[j, 0]
        n=0
        while (n<(len(full)-n_samples)):
            clip = full[n: (n+n_samples)]
            y = df[j, 1]
            new_row = np.array([clip, y], dtype=object)
            a_clip = np.vstack([a_clip, new_row])
            n+=int(n_samples/2)
    return a_clip[1:]
          

In [9]:
test = clip_stft(test, 128)
validation = clip_stft(validation, 128)
training = clip_stft(training, 128)

In [10]:
test[0,0].shape

(128, 513)

In [11]:
from torch.utils.data import Dataset, DataLoader

#Class for the creation of torch manageble datasets, with Format one can select the desired input column 
class DataAudio(Dataset):

    def __init__(self, data, transform=None):
        self.x = data[:,0]
        self.y = data[:,1]
        self.transform = transform

    def __len__(self):
       
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        if self.transform:
            x = self.transform(x)
            #x = x.unsqueeze(0)
            
        return x, y

In [12]:
transforms = Compose([
    ToTensor(), #this converts numpy or Pil image to torch tensor and normalizes it in 0, 1
])

In [13]:
#Creation of torch suited dataset classes  (change string 'mel' to select desired Format)
test_dataset = DataAudio(data=test, transform=transforms)
validation_dataset = DataAudio(data=validation, transform=transforms)
training_dataset = DataAudio(data=training, transform=transforms)

In [1]:
test_dataset

NameError: name 'test_dataset' is not defined

In [14]:
#Creation of dataloader classes
batch_size = 64
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True, num_workers=os.cpu_count())
validation_dataloader = DataLoader(validation_dataset, batch_size, shuffle=True, num_workers=os.cpu_count())
training_dataloader = DataLoader(training_dataset, batch_size, shuffle=True, num_workers=os.cpu_count())

In [15]:
test_dataloader.dataset.x

array([array([[2.1080021e-04, 2.0972929e-04, 2.0654705e-04, ..., 8.1103675e-09,
               8.0495903e-09, 8.0293825e-09],
              [3.8682382e+00, 5.6146770e+00, 1.0434763e+01, ..., 3.2188848e-07,
               7.4267308e-07, 3.5079273e-07],
              [1.7605966e+00, 4.1246676e+00, 6.5461721e+00, ..., 1.7453565e-06,
               4.9375876e-07, 1.2367860e-07],
              ...,
              [2.2091374e+00, 1.3930146e+00, 1.0820771e+01, ..., 1.9613076e-06,
               2.2745394e-06, 6.6840818e-07],
              [3.5874403e+00, 5.3710556e+00, 1.9456934e+01, ..., 3.6745055e-07,
               1.0445683e-06, 1.6783180e-06],
              [2.8450794e+00, 5.4273024e+00, 2.3327595e+01, ..., 8.4913364e-07,
               9.5739790e-07, 2.4902607e-07]], dtype=float32)                  ,
       array([[2.1542284e+00, 3.2158666e+00, 2.2255291e+01, ..., 9.5538928e-07,
               3.7759537e-07, 1.0313137e-06],
              [1.1447760e+00, 9.4032307e+00, 3.8288792e+01, ...,

In [16]:
for batch_x, batch_y in test_dataloader:
        print(batch_x)
        print(batch_y)

In [24]:
# Remember to add dropout layers 

class NNET1(nn.Module):
    
    def __init__(self):
        super(NNET1, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=128,kernel_size=(4,513)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,1)),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(4,1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,1)),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(4,1)),
            nn.ReLU(),
        )

        # Input of fc1 is 256

        self.fc = nn.Sequential(
            nn.Linear(256, 300),
            nn.ReLU(),
            nn.Linear(300, 150),
            nn.ReLU(),
            nn.Linear(150, 10),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        x = self.conv(x)
        max_pool = F.max_pool2d(x, kernel_size=(26,1))
        avg_pool = F.avg_pool2d(x, kernel_size=(26,1))
        x = max_pool + avg_pool
        x = self.fc(x.view(-1, 256))
        
        return x

In [25]:
#Remember to add dropout

class NNET2(nn.Module):
        
    def __init__(self):
        super(NNET2, self).__init__()
        
        
        self.c1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=256,kernel_size=(4,513)),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )

        self.c2 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(4, 1),padding=(2,0)),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )

        self.c3 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(4, 1),padding=(1,0)),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(256, 300),
            nn.ReLU(),
            nn.Linear(300, 150),
            nn.ReLU(),
            nn.Linear(150, 10),
            nn.Softmax(dim=1)
        )

    def forward(self,x):
        
        c1 = self.c1(x)
        c2 = self.c2(c1)
        c3 = self.c3(c2)
        x = c1 + c3
        max_pool = F.max_pool2d(x, kernel_size=(125,1))
        avg_pool = F.avg_pool2d(x, kernel_size=(125,1))
        x = max_pool + avg_pool
        x = self.fc(x.view(-1, 256))
        return x 




In [26]:
# Fake spectrogram 128x513
x = torch.randn(1, 1, 128, 513)

# Create model
model = NNET2()


# Forward pass
output = model(x)

# Print output shape
print(output.shape)

torch.Size([1, 10])


In [27]:
from torch.optim import SGD, Adam, Adadelta
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
# Create model
model = NNET2()
opt = Adadelta(model.parameters())

loss_fn = CrossEntropyLoss()

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
epochs=10
best_val = np.inf
for epoch in range(epochs):
    model.train()
    print(f"Epoch: {epoch+1}")
    iterator = tqdm(test_dataloader)
    for batch_x, batch_y in iterator:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        y_pred = model(batch_x)

        loss = loss_fn(y_pred, batch_y)

        opt.zero_grad()
        loss.backward()
        opt.step()
        iterator.set_description(f"Train loss: {loss.detach().cpu().numpy()}")

    model.eval()
    with torch.no_grad():
        predictions = []
        true = []
        for batch_x, batch_y in tqdm(test_dataloader):
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            y_pred = model(batch_x)

            predictions.append(y_pred)
            true.append(batch_y)
        predictions = torch.cat(predictions, axis=0)
        true = torch.cat(true, axis=0)
        val_loss = loss_fn(predictions, true)
        val_acc = (torch.sigmoid(predictions).round() == true).float().mean()
        print(f"loss: {val_loss}, accuracy: {val_acc}")
    
    if val_loss < best_val:
        print("Saved Model")
        torch.save(model.state_dict(), "model.pt")
        best_val = val_loss

Epoch: 1


  0%|          | 0/238 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_curve, roc_auc_score

def evaluate_network(dataloader, model, data_split):
    model.eval()
    with torch.no_grad():
        predictions = []
        true = []
        for batch_x, batch_y in tqdm(dataloader):
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            y_pred = model(batch_x)

            predictions.append(y_pred)
            true.append(batch_y)
        predictions = torch.cat(predictions, axis=0)
        true = torch.cat(true, axis=0)
        loss = loss_fn(predictions, true).detach().cpu().numpy()
        predictions = torch.sigmoid(predictions).detach().cpu().numpy()
        true = true.detach().cpu().numpy()

        fpr, tpr, thresholds = roc_curve(true, predictions)
        auc = roc_auc_score(true, predictions)
        predictions = predictions.round()
        precision, recall, fscore, _= precision_recall_fscore_support(true, predictions, average='binary')
        accuracy = accuracy_score(true, predictions)

        print(f"{data_split} loss: {loss}, accuracy: {accuracy}, precision: {precision}, recall: {recall}, f1: {fscore}, roc_auc: {auc}")

        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'{data_split} receiver operating characteristic (ROC)')
        plt.legend(loc="lower right")

In [ ]:
model = NNET2()
model.load_state_dict(torch.load("model.pt"))

evaluate_network(test_dataloader, model, "Test Dataset")